In [261]:
import numpy as np
from scipy.ndimage.measurements import label
import threading

In [5]:
VOL_UNAVAILABLE = -1
VOL_BORDER = 1
VOL_INTERIOR = -2
VOL_EMPTY = 0 

The following are just dummy data sets to run the algorithm

In [20]:
article_list = [(
    "cube_1",
    2,
    [(
        1,
        10,
        10,
        10,
        1
    )]
),
(
    "cube_2",
    2,
    [(
        1,
        15,
        15,
        15,
        2
    )]
),
(
    "cuboid_1",
    1,
    [(
        1,
        15,
        30,
        30,
        2
    )]
),
(
    "cuboid_2",
    1,
    [(
        1,
        15,
        30,
        30,
        1
    ),
    (
        2,
        5,
        5,
        10,
        1
    )]
)]

In [485]:
volume_space = np.full((40,40,40), VOL_EMPTY, dtype=int)
volume_space[0,:,:] = VOL_UNAVAILABLE
volume_space[:,0:2,:] = VOL_UNAVAILABLE
volume_space[:,:,0] = VOL_UNAVAILABLE

## General fit

### Overall volume fit

In [543]:
def calculate_package_volume(package):
    '''
    Returns volume in cm cubed for package
    '''
    return np.prod(package)

In [544]:
def find_total_package_volume(article_list):
    '''
    Establishes total volume of packages in list in cm cubed.
    Loops through each article in article_list, and each package in article.
    Multiplies length, height and width (elements 1-3).
    Sums up individual products and returns them as int
    '''
    return sum([calculate_package_volume(package[1:4]) for article in article_list for package in article[2]])

In [545]:
find_total_package_volume(article_list)

31625

In [230]:
def find_available_space(volume_space):
    '''
    Returns total available volume in cm cubed by subtracting non-zero values from total array size
    '''
    return volume_space.size-np.count_nonzero(volume_space)

In [93]:
find_available_space(volume_space)

32500

In [94]:
def is_space_sufficient(article_list, volume_space):
    '''
    Returns True if package volume is smaller than or equal to available volume
    '''
    return find_total_package_volume(article_list) <= find_available_space(volume_space)

In [471]:
is_space_sufficient(article_list, volume_space)

True

### Fit by longest dimension

In [231]:
def find_longest_package_dimension(article_list):
    '''
    Finds single longest dimension in cm among all packages.
    '''
    return max([max(package[1:4]) for article in article_list for package in article[2]])

In [97]:
find_longest_package_dimension(article_list)

30

In [110]:
def binarize_space(volume_space):
    '''
    Sets empty space to 1 and rest to 0.
    Needed to be able to identify clusters.
    '''
    arbitrary_constant = max(VOL_UNAVAILABLE, VOL_BORDER, VOL_INTERIOR, VOL_EMPTY) + 1
    first_step = np.where(volume_space != VOL_EMPTY, arbitrary_constant, volume_space)
    second_step = np.where(first_step == VOL_EMPTY, 1, first_step)
    return np.where(second_step == arbitrary_constant, 0, second_step)

In [149]:
# Linear structures in all three axes
structure_y = [
    [[0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]],
    [[0, 1, 0],
     [0, 1, 0],
     [0, 1, 0]],
    [[0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]]]
structure_x = np.rot90(structure_x, axes=(1,2))
structure_z = np.rot90(structure_x, axes=(0,1))
structures = [structure_x, structure_y, structure_z]

In [232]:
def find_longest_space_dimension(volume_space):
    '''
    Determines the longest continuous stretch of empty space in all three dimensions.
    '''
    # Run along each dimension
    results = []
    for structure in structures:
        # Binarize the space, then label along given dimension
        dim_result = label(input = binarize_space(volume_space), structure=structure)[0]
        # Turn into a 1-dimensional array
        dim_result = np.reshape(dim_result, dim_result.size)
        # Remove zeros
        dim_result = np.delete(dim_result, np.where(dim_result == 0))
        # Find most frequent occurrence
        most_freq = np.bincount(dim_result).argmax()
        # Return number of occurrences, which is equal to dimension in cm
        results.append(np.count_nonzero(dim_result == most_freq))
    # Return largest value from the three dimensions
    return max(results)

In [472]:
def is_longest_dimension_sufficient(article_list, volume_space):
    '''
    Returns True if longest package dimension is smaller than or equal to longest space dimension
    '''
    return find_longest_package_dimension <= find_longest_space_dimension(volume_space)

## Penalizer/Scorer

In [233]:
def find_empty_space(volume_space):
    '''
    Identifies contiguous pockets of empty space.
    Returns number of empty spaces, their total volume and the volume of the largest empty space.
    '''
    # Set up structure - this treats as one contiguous space when...
    # ...there are diagonal gaps as it might make the stack unstable
    structure = np.ones((3,3,3), dtype=int)
    # Binarize the space, then label pockets
    dim_result = label(input = binarize_space(volume_space), structure=structure)[0]
    # How many empty spaces are there?
    number_spaces = np.max(dim_result)
    # What's the total volume of empty space?
    empty_volume = np.count_nonzero(dim_result)
    # What's the volume of the largest empty space?
    largest_volume = np.count_nonzero(dim_result == number_spaces)
    return number_spaces, empty_volume, largest_volume

In [183]:
find_empty_space(volume_space)

(2, 20400, 18000)

In [234]:
def score_space(volume_space, article_list):
    '''
    This function assigns a score to the empty space left by the optimization algorithm.
    There is a penalty for having more empty spaces as well as a relatively smaller largest empty space.
    '''
    # Get number of empty spaces, their total volume and the volume of the largest empty space
    number_spaces, empty_volume, largest_volume = find_empty_space(volume_space)
    # Get maximum possible empty space (assumes everything can be stacked perfectly)
    max_largest_volume = find_available_space(volume_space) - find_total_package_volume(article_list)
    '''INITIAL IDEA:
    - Linear penalty for each empty space
    - Log penalty for how much largest empty space is smaller than maximum possible
    '''
    score = number_spaces + np.log(max_largest_volume - largest_volume)
    return score

## Optimizer

### Sort packages

In [345]:
def sort_packages(article_list, sort_by="volume", direction="descending"):
    '''
    Returns 2-dimensional list of packages sorted by sort_by in the shape of
    [(article_code, article_id, package_id)]
    '''
    # Initialize list
    return_list = []
    # Loop through articles
    for article in article_list:
        # If articles exist more than once, loop here too
        for article_id in range(article[1]):
            # Loop through packages
            for package in article[2]:
                # So far only sorting by volume has been implemented.
                if sort_by == "volume":      
                    return_list.append((article[0], article_id+1, package[0], calculate_package_volume(package)))
    # Turn into a numpy array for easier sorting
    return_list = np.array(return_list)
    # Sort by volume (4th column)
    return_list = return_list[return_list[:,3].astype(int).argsort()]
    # This gives an ascending order. Reverse if needed
    if direction == "descending":
        return_list = np.flip(return_list, axis=0)
    # Remove volume column
    return return_list[:,:3]

In [346]:
sort_packages(article_list)

array([['rectangles_2', '1', '1'],
       ['rectangle_1', '1', '1'],
       ['cube_2', '2', '1'],
       ['cube_2', '1', '1'],
       ['cube_1', '2', '1'],
       ['cube_1', '1', '1'],
       ['rectangles_2', '1', '2']], dtype='<U21')

We could easily have other sorting functions, or some randomness:

In [382]:
def hash_list(article_list):
    '''
    Creates an element-wise list of hashes
    '''
    return [hash(article.tobytes()) for article in article_list]

In [456]:
def generate_package_lists(article_list, sorters = ["volume|ascending", "volume|ascending", "volume|descending"], random_lists = 10):
    '''
    Returns list of package lists for the optimizer.
    Lists can be pre-defined or randomized.
    '''
    
    '''
    IMPROVEMENT OPTION:
    - if random_lists is larger than maximum number of permutations, just use permutations directly
    '''
    
    
    package_lists = []
    # First generate pre-defined lists
    sorters = set(sorters)
    for sorter in sorters:
        # Split by delimiter
        sort_by, direction = sorter.split("|")
        package_lists.append(sort_packages(article_list, sort_by=sort_by, direction=direction))
    
    # Add random lists
    # Create starting point if there is none, i.e. no pre-defined lists
    if len(package_lists) == 0:
        starter_list = sort_packages(article_list)
    else:
        starter_list = np.copy(package_lists[0])
    
    # What is the actual maximum number of unique lists?
    max_permut = np.math.factorial(len(starter_list))

    # Now add as many random lists as needed    
    while True:
        # Shuffle starter list and copy data
        np.random.shuffle(starter_list)
        new_list = np.copy(starter_list)
        # Check if this particular permutation exists already by looking at hashes
        if hash_list([new_list])[0] not in hash_list(package_lists):
            package_lists.append(new_list)
        else:
            print("duplicate")
        # Check if length requirement (smaller of pre-defined lists + random_lists and max_permut) is fulfilled
        if len(package_lists) == min(len(sorters) + random_lists, max_permut):
            break
    return package_lists

In [464]:
a = generate_package_lists(article_list)[0]

### Optimizer function

In [566]:
INSUFFICIENT_SPACE = "Not enough space for packages in chosen trunk."
INSUFFICIENT_DIMENSION = "At least one package dimension exceeds trunk dimension."
OPT_INSUFFICIENT_SPACE = "Optimizer could not place this package (internal return code)."

In [567]:
#TODO
def generate_optimizer(article_list, volume_space):
    '''
    Main function to run this module.
    First checks if packages can fit at all, then calls optimizer.
    '''
    # Check if package volume is smaller than or equal to available space, otherwise return error
    if not is_space_sufficient(article_list, volume_space):
        return INSUFFICIENT_SPACE
    # Check if longest package dimension is smaller than or equal to longest space dimension, otherwise return error
    if not is_longest_dimension_sufficient(article_list, volume_space):
        return INSUFFICIENT_DIMENSION
    # Generate list of package lists
    package_lists = generate_package_lists(article_list)
    # Call optimizer function for each package list
    

In [631]:
#TODO
def place_package(package, x, y, z, volume_space):
    '''
    Attempt to place next package.
    If successful, return filled volume_space,
    if not, return false.
    '''
    # Unpack package dimensions
    package_x, package_y, package_z = package[0], package[1], package[2]
    package_volume = calculate_package_volume(package)
    # Check available space - there need to be as many zeros as the package_volume for it to fit
    available_space = volume_space[x:x+package_x, y:y+package_y, z:z+package_z]
    # If the package cannot be placed, return
    if package_volume != available_space.size - np.count_nonzero(available_space):
        return OPT_INSUFFICIENT_SPACE
    # Otherwise populate the array
    # Surfaces first
    # z-plain
    volume_space[x:x+package_x, y:y+package_y, z] = VOL_BORDER
    volume_space[x:x+package_x, y:y+package_y, z+package_z] = VOL_BORDER
    # y-plain
    volume_space[x:x+package_x, y, z:z+package_z] = VOL_BORDER
    volume_space[x:x+package_x, y+package_y, z:z+package_z] = VOL_BORDER
    '''
    # x-plain
    volume_space[x, y:y+package_y, z:z+package_z] = VOL_BORDER
    volume_space[x+package_x, y:y+package_y, z:z+package_z] = VOL_BORDER
    '''
    # Fill interior area
    #volume_space[x+1:x+package_x-1, y+1:y+package_y-1, z+1:z+package_z-1] = VOL_INTERIOR
    
    return volume_space

In [629]:
vol_space = np.full((10,10,10), VOL_EMPTY, dtype=int)
vol_space[0,0:2,0:2] = VOL_UNAVAILABLE
vol_space[0,0,2] = VOL_UNAVAILABLE
vol_space

array([[[-1, -1, -1,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]

In [630]:
place_package([4,4,3], 0, 1, 2, vol_space)

array([[[-1, -1, -1,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1,  1,  1,  1,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  1,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  1,  1,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  1,  1,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]

In [519]:
#TODO
def optimizer(package_list, volume_space):
    '''
    Recursively places one package after another into available space.
    '''
    # Package counter variable
    package_counter = 0
    # Find starting coordinates
    locator = np.where(volume_space == VOL_EMPTY)
    x, y, z = locator[0][0], locator[1][0], locator[2][0]
    # Attempt to place next package in list
    
    return x, y, z

In [517]:
optimizer(a, vol_space)

((array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
  array([1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2]),
  array([2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2])),
 0,
 1,
 2)

In [497]:
np.rot90(vol_space, 3, axes=(1,2))

array([[[ 0, -1, -1],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]])

### Set up threading